In [1]:
# Import Libraries

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [2]:
#PINECONE_API_KEY = "c3421ec9-e32d-458f-94d4-1a149f5ab0c5"
#PINECONE_API_ENV = "gcp-starter"

In [2]:
def load_pdf(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf("E:/Medical_Chatbot/data")

In [7]:
len(extracted_data)

198

In [14]:
# Second Step- Text Chunks

In [15]:
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap= 20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [16]:
text_chunks= text_split(extracted_data)
print("length of the chunk:", len(text_chunks))

length of the chunk: 446


In [17]:
print(text_chunks[8].page_content)

5 KEEP GOING 
Resources                                                                                                                               92 
Worksheets                                                              102 
WHAT’S INSIDE 
This booklet will 
help you learn about exercise and physical activity as you age. 
Get Fit for Life  | 5


In [8]:
# Step 3: Vector Embedding

In [18]:
# download embedding model

def download_hugging_face_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [19]:
embeddings=download_hugging_face_embeddings()

e:\Medical_Chatbot\mdenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# try with example--> vector representation of hello word

query_result= embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
#persist_dirctory="db"
#vectordb= Chroma.from_documents(documents=extracted_data, embeddings=embeddings, persist_dirctory=persist_dirctor)

In [23]:
import os
import pinecone
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key="c3421ec9-e32d-458f-94d4-1a149f5ab0c5")

index_name="medical-chatbot"

# Check if the index exists
if index_name in pc.list_indexes():
    pc.deindex(index_name)

In [27]:
# Create embeddings for each text chunk
vectors = embeddings.embed_documents([t.page_content for t in text_chunks])

# Upsert the embeddings into Pinecone
doc_ids = [f"{t.lc_id}_{i}" for i, t in enumerate(text_chunks)]  # Using 'lc_id' as the document ID

In [28]:
items = [{"id": str(doc_id), "values": embedding} for doc_id, embedding in zip(doc_ids, vectors)]


In [30]:
# Instantiate an Index object
index = pc.Index(name="medical-chatbot")

In [31]:
# Upsert vectors
index.upsert(vectors=items)

{'upserted_count': 446}

In [33]:
def get_similiar_docs(query, k=1, score=False):
    if score:
        similiar_docs= index.query(queries=query, top_k=k)
    else:
        similiar_docs= index.query(queries=query, top_k=k)
    return similiar_docs


In [35]:
# Assuming 'embeddings' is your text embedding model and 'index' is your Pinecone index
query_text = "LIFTING WEIGHTS"
query_vector = embeddings.embed_documents([query_text])
similiar_docs = index.query(queries=query_vector, top_k=5)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Length': '90', 'content-type': 'text/plain', 'date': 'Mon, 19 Feb 2024 17:59:20 GMT', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: queries[182]: invalid value 0.0092676877975463867 for type type.googleapis.com/QueryVector


In [20]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00447,
 'namespaces': {'': {'vector_count': 447}},
 'total_vector_count': 447}

In [21]:
# Your list of document IDs
doc_ids = [t.lc_id for t in text_chunks]

# Convert the list to a set, which removes duplicates
unique_ids = set(doc_ids)

# Check if all IDs are unique
if len(doc_ids) == len(unique_ids):
    print("All document IDs are unique.")
else:
    print("Some document IDs are not unique.")


Some document IDs are not unique.
